# 0. Setup

In [2]:
!pip install pandasql

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.2/613.2 kB 14.1 MB/s eta 0:00:0000:01
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=e84c8f0230c9fa3f3997a04491818da0f0146a61f5f83f5757cac628a54ed862
  Stored in directory: /home/codespace/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [6]:
import pandas as pd
from pandasql import sqldf

# Carregar dados
df = pd.read_csv('credit.csv')

# Função utilitária para executar SQL
pysqldf = lambda q: sqldf(q, globals())


# Análise Exploratória de Dados (EDA) com SQL
## Visão Geral do Projeto

Bem-vindo ao projeto de Análise Exploratória de Dados, onde o SQL é utilizado para explorar, analisar e visualizar dados financeiros. Este projeto tem como objetivo fornecer insights sobre um conjunto de dados de clientes, que inclui informações detalhadas como idade, sexo, dependentes, escolaridade, estado civil, salário anual, tipo de cartão, entre outros. 

## Objetivos
Este notebook é dedicado a conduzir uma análise exploratória abrangente, utilizando a linguagem SQL para:

* Entender as Características Básicas dos Dados: Estatísticas descritivas para ter uma visão geral das características fundamentais do conjunto de dados.
* Analisar Relações e Correlações: Como diferentes variáveis interagem entre si, buscando entender as relações e correlações presentes nos dados.
* Identificar Tendências e Padrões: Detecção de tendências, padrões e anomalias, entendendo como certos atributos variam ao longo do tempo ou entre diferentes segmentos.

## Metodologia
O pandasql será utlizado, uma biblioteca que permite executar consultas SQL diretamente em DataFrames do Pandas, para realizar a EDA. Isso proporciona a flexibilidade da linguagem SQL, juntamente com o poder do Pandas para manipulação de dados em Python.

# Seção 1: Estatísticas Básicas

## 1.1. Estatísticas Descritivas para as Colunas Numéricas

In [9]:
query = """
SELECT
    ROUND(AVG(idade), 2) as idade_media,
    ROUND(AVG(dependentes), 2) as dependentes_media,
    ROUND(AVG(qtd_produtos), 2) as qtd_produtos_media,
    ROUND(AVG(iteracoes_12m), 2) as iteracoes_12m_media,
    ROUND(AVG(meses_inativo_12m), 2) as meses_inativo_12m_media,
    ROUND(AVG(limite_credito), 2) as limite_credito_media,
    ROUND(AVG(valor_transacoes_12m), 2) as valor_transacoes_12m_media,
    ROUND(AVG(qtd_transacoes_12m), 2) as qtd_transacoes_12m_media
FROM df;
"""
pysqldf(query)


,idade_media,dependentes_media,qtd_produtos_media,iteracoes_12m_media,meses_inativo_12m_media,limite_credito_media,valor_transacoes_12m_media,qtd_transacoes_12m_media
0,45.91,2.09,4.36,2.57,2.27,9072.72,1820.33,41.7


## 1.2 Distribuição de Frequência

In [10]:
query = """
SELECT 'sexo' as categoria, sexo as valor, COUNT(*) as frequencia FROM df GROUP BY sexo
UNION ALL
SELECT 'escolaridade', escolaridade, COUNT(*) FROM df GROUP BY escolaridade
UNION ALL
SELECT 'estado_civil', estado_civil, COUNT(*) FROM df GROUP BY estado_civil
UNION ALL
SELECT 'tipo_cartao', tipo_cartao, COUNT(*) FROM df GROUP BY tipo_cartao;
"""
pysqldf(query)


,categoria,valor,frequencia
0,sexo,F,1001
1,sexo,M,1563
2,escolaridade,doutorado,235
3,escolaridade,ensino medio,545
4,escolaridade,graduacao,261
5,escolaridade,mestrado,810
6,escolaridade,na,346
7,escolaridade,sem educacao formal,367
8,estado_civil,casado,1521
9,estado_civil,divorciado,161


# Seção 2: Análise de Relações e Correlações

## 2.1 Relação entre Salário Anual e Escolaridade

In [17]:
query = """
SELECT escolaridade, (salario_anual) as salario_medio
FROM df
GROUP BY escolaridade;
"""
pysqldf(query)


,escolaridade,salario_medio
0,doutorado,$80K - $120K
1,ensino medio,$60K - $80K
2,graduacao,$80K - $120K
3,mestrado,menos que $40K
4,na,$120K +
5,sem educacao formal,$60K - $80K


## 2.2 Inatividade e Limite de Crédito

In [19]:
query = """
SELECT meses_inativo_12m, ROUND(AVG(limite_credito), 2) as limite_medio
FROM df
GROUP BY meses_inativo_12m;
"""
pysqldf(query)


,meses_inativo_12m,limite_medio
0,0,8914.17
1,1,8890.53
2,2,9446.43
3,3,8913.29
4,4,9347.02
5,5,7306.79
6,6,8402.75


## 2.3 Inatividade e Interações

In [34]:
query = """
SELECT 
    meses_inativo_12m,
    ROUND(AVG(iteracoes_12m), 2) as iteracoes_12m_media
FROM df
GROUP BY meses_inativo_12m;
"""
pysqldf(query)


,meses_inativo_12m,iteracoes_12m_media
0,0,2.17
1,1,2.49
2,2,2.60
3,3,2.58
4,4,2.73
5,5,2.69
6,6,2.59


## 2.4 Tendência de Crédito ao Longo da Idade

In [28]:
query = """
SELECT 
    (idade / 10) * 10 as faixa_etaria,
    ROUND(AVG(limite_credito), 2) as limite_credito_medio
FROM df
GROUP BY faixa_etaria
ORDER BY faixa_etaria;
"""
pysqldf(query)


,faixa_etaria,limite_credito_medio
0,20,4421.81
1,30,7953.84
2,40,11533.69
3,50,9372.54
4,60,5494.68
5,70,3860.57


## 2.5 Análise da Inatividade por Faixa de Salário

In [32]:
query = """
SELECT 
    A.salario_anual,
    A.meses_inativo_12m_media,
    B.meses_inativo_12m,
    B.iteracoes_12m_media
FROM 
    (SELECT 
        salario_anual,
        ROUND(AVG(meses_inativo_12m), 2) as meses_inativo_12m_media
    FROM df
    GROUP BY salario_anual) A
CROSS JOIN 
    (SELECT 
        meses_inativo_12m,
        ROUND(AVG(iteracoes_12m), 2) as iteracoes_12m_media
    FROM df
    GROUP BY meses_inativo_12m) B
"""
pysqldf(query)


,salario_anual,meses_inativo_12m_media,meses_inativo_12m,iteracoes_12m_media
0,$120K +,2.22,0,2.17
1,$120K +,2.22,1,2.49
2,$120K +,2.22,2,2.60
3,$120K +,2.22,3,2.58
4,$120K +,2.22,4,2.73
5,$120K +,2.22,5,2.69
6,$120K +,2.22,6,2.59
7,$40K - $60K,2.21,0,2.17
8,$40K - $60K,2.21,1,2.49
9,$40K - $60K,2.21,2,2.60


## 2.6 Variação do Número de Produtos por Escolaridade

In [33]:
query = """
SELECT 
    escolaridade,
    ROUND(AVG(qtd_produtos), 2) as qtd_produtos_media
FROM df
GROUP BY escolaridade;
"""
pysqldf(query)


,escolaridade,qtd_produtos_media
0,doutorado,4.27
1,ensino medio,4.33
2,graduacao,4.22
3,mestrado,4.37
4,na,4.51
5,sem educacao formal,4.37


# 3. Conclusão

Após uma análise do conjunto de dados de clientes, algumas conclusões e insights podem ser extraídos.


### Características Gerais dos Clientes
* A idade média dos clientes é de aproximadamente 45 anos, com uma média de 2 dependentes. Isso sugere que muitos clientes estão possivelmente em uma fase de vida estável com famílias constituídas.
* Em média, os clientes possuem cerca de 4 produtos, indicando um engajamento significativo com os serviços oferecidos. A média de interações nos últimos 12 meses é de aproximadamente 2.6 vezes, com uma inatividade média de cerca de 2.3 meses.

### Demografia e Educação
* A distribuição por gênero mostra uma maior proporção de homens (1563) em comparação com mulheres (1001).
* Quanto à escolaridade, uma grande parte dos clientes possui mestrado, seguido por ensino médio e doutorado. Isso reflete um nível educacional diversificado entre os clientes.
* Em termos de salário médio, indivíduos com doutorado tendem a ter salários na faixa de $80K - $120K, enquanto aqueles com mestrado estão majoritariamente na faixa de menos de $40K. Interessantemente, clientes sem educação formal ou com a classificação "na" tendem a ter salários mais altos, possivelmente indicando empresários ou profissionais autônomos.

### Relações Financeiras
* O limite de crédito médio tende a aumentar com a idade, atingindo um pico na faixa dos 40 anos e diminuindo a partir daí. Isso pode estar relacionado ao aumento da estabilidade financeira e da capacidade de crédito à medida que os clientes envelhecem.
* A análise por faixa salarial revela que clientes com salários mais altos ($120K +) tendem a ter um pouco mais de inatividade e uma média de interações similar às outras faixas salariais.

### Produtos e Serviços
* A quantidade média de produtos varia ligeiramente com a escolaridade, sendo um pouco maior para clientes sem educação formal e para aqueles com a classificação "na", o que pode sugerir uma abordagem mais diversificada ou experimental em relação aos produtos financeiros.

### Implicações e Recomendações
* As instituições financeiras podem aproveitar esses insights para personalizar suas ofertas e comunicações, especialmente considerando as diferenças de interação e inatividade com base em salário e idade.
* Programas de fidelidade ou incentivos podem ser especialmente focados em faixas etárias específicas, onde o engajamento com produtos é mais alto.
* Estratégias de marketing e educação financeira podem ser adaptadas para atender às necessidades específicas dos diferentes segmentos educacionais.